In [6]:
import pandas as pd, numpy as np
import matplotlib, seaborn as sns
from sklearn.metrics import mean_squared_error as mse
matplotlib.use('Qt5Agg')
import matplotlib.pyplot as plt
plt.rcParams["figure.figsize"] = (7,3)
plt.rcParams['figure.constrained_layout.use'] = True

In [2]:
# load data
mydateparser = lambda x: pd.datetime.strptime(str(x), '%Y-%m-%d %H:%M:%S')
btc_series = pd.read_excel('Data/BTC_closing.xlsx',squeeze=True, parse_dates=[0], index_col=0, date_parser=mydateparser)

In [8]:
plt.plot(btc_series)
plt.show()

In [24]:
# crop data: 2015-08-07 to 2020-06-26
start_date = '2015-08-07'
end_date = '2020-06-26'
btc_series = btc_series[start_date: end_date]

In [25]:
# split data into train and test
train_size = int(len(btc_series) * 0.8)
train, test = btc_series[:train_size], btc_series[train_size:]
print('train:', len(train), '\ntest:', len(test), '\nSUM:', len(train) + len(test), '\nlen btc_series:', len(btc_series))

train: 1428 
test: 358 
SUM: 1786 
len btc_series: 1786


In [26]:
# NaivePrediction: Walk forward validation (yt = yt-1)
history = [x for x in train]
predictions = list()

for i in range(len(test)):
    yhat = history[-1]
    predictions.append(yhat)
    history.append(test[i])

In [27]:
# make series for plotting
index_pred = np.arange(len(train) + 1, len(btc_series) + 1)
predictions_series = pd.Series(predictions, index=index_pred)
actual_series = pd.Series(test.values, index=index_pred)

# plot
plt.plot(train.values, label='training')
plt.plot(actual_series, label='actual')
plt.plot(predictions_series, label='prediction', color='darkgreen')
plt.legend(loc='upper left')
plt.show()

In [28]:
# calculate forecast accuracy
def prediction_accuracy(predictions, actual):
    # MAPE: Mean absolute percentage error
    mape = np.mean(np.abs(predictions - actual) / np.abs(actual))
    # MPE: Mean percentage error
    mpe = np.mean((predictions - actual) / actual)
    # ME: Mean error
    me = np.mean(predictions - actual)
    # MAE: Mean absolute error
    mae = np.mean(np.abs(predictions - actual))
    # RSME: Root mean squared error
    rmse = np.sqrt(mse(predictions, actual))
    # CORR: Correlation between Prediction and Actual
    corr = np.corrcoef(predictions, actual)[0,1]
    
    print('Mean absolute percentage error:\t', mape,\
             '\nMean percentage error:\t\t', mpe,\
             '\nMean error:\t\t\t', me, \
             '\nMean absolute error:\t\t', mae, \
             '\nRoot mean squared error:\t', rmse, \
             '\nCorrelation coefficient:\t', corr, '\n')
    return

prediction_accuracy(np.array(predictions), test.values)


Mean absolute percentage error:	 0.0252483316221213 
Mean percentage error:		 0.001516064858912412 
Mean error:			 5.733296089385476 
Mean absolute error:		 214.59826815642455 
Root mean squared error:	 338.61646158924094 
Correlation coefficient:	 0.971534824695731 

